## Lista De Exercícios

Grupo:
- Gregory Filipe Lira da Silva
- Danilo Henrique da Silva Santana
- Dayvison Gomes de Oliveira

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import random
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs
from copy import deepcopy
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

import tensorflow as tf

from mlxtend.plotting import plot_decision_regions

from neural_network import *

%matplotlib inline

### Utils

In [ ]:
def __softmax(x):
    exp = np.exp(x)
    return exp / np.sum(exp, axis=1, keepdims=True)

def __compute_meshgrid(x, y):
    x_min, x_max, y_min, y_max = x[:, 0].min(), x[:, 0].max(), x[:, 1].min(), x[:, 1].max()
    x1, x2 = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    x_mesh = np.array([x1.ravel(), x2.ravel()]).T
    return x1, x2, x_mesh

def classification_predictions(x, y, is_binary, nn=None, threshold=0.0, figsize=(12,6), s=15, cmap=plt.cm.viridis):
    plt.figure(figsize=figsize)
    ax = plt.subplot(1, 2, 1)
    plt.scatter(x[:, 0], x[:, 1], c=list(np.array(y).ravel()), s=s, cmap=cmap)

    if nn is not None:
        plt.subplot(1, 2, 2, sharex=ax, sharey=ax)

        x1, x2, x_mesh = __compute_meshgrid(x, y)
        y_mesh = nn.predict(x_mesh)
        y_mesh = np.where(y_mesh <= threshold, 0, 1) if is_binary else np.argmax(__softmax(y_mesh), axis=1)

        plt.scatter(x[:, 0], x[:, 1], c=list(np.array(y).ravel()), s=s, cmap=cmap)
        plt.contourf(x1, x2, y_mesh.reshape(x1.shape), cmap=cmap, alpha=0.5)

        
def make_log10(n_samples, x_min, x_max, noise=0.0, random_state=None):
    np.random.seed(random_state)
    x = np.logspace(np.log10(x_min), np.log10(x_max), n_samples)
    y = np.log10(x) + 2*noise*np.random.random(n_samples) - noise
    return x.reshape(-1,1), y.reshape(-1,1)

## Questão 1

1) A representação de uma determinada mensagem digital ternária, isto é formada por três bits,
forma um cubo cujos vértices correspondem a mesma representação digital. Supondo que ao
transmitirmos esta mensagem a mesma possa ser contaminada por ruído formado em torno de
cada vértice uma nuvem esférica de valores aleatórios com raio máximo é 0.1. Formule este
problema como um problema de classificação de padrões e treine uma Rede Perceptron de
Rosenblatt (Perceptron de camada única) para atuar como classificador/decodificador. Para
solução do problema defina antes um conjunto de treinamento e um conjunto de validação

In [ ]:
x = np.array([[0,0,0], [0,0,1], [0,1,0], [0,1,1], [1,0,0], [1,0,1], [1,1,0],[1,1,1]])
print(x.shape)
classes = [ i for i in range(x.shape[0])] # para organizar e colocar cada um das 8 classes
data = pd.DataFrame(np.column_stack((x,classes)), columns=['x1','x2','x3','class'])
data

In [ ]:
def gerar_dados(quantidade, base):
    dados_gerados = deepcopy(base)
    valores = dados_gerados[['x1','x2','x3']].values
    
    for i in range(quantidade):
        ruido = np.array([np.random.uniform(-0.1,0.1) for i in range(x.shape[0]*x.shape[1])]).reshape(8,3)
        novos_valores = valores + ruido
        for k in range(len(novos_valores)):
            #adicionando novos dados no final do dataset com o ruido
            dados_gerados.loc[len(dados_gerados)] = [novos_valores[k][0],novos_valores[k][1],novos_valores[k][2],k]
            
    return dados_gerados

In [ ]:
dados_novos = gerar_dados(500,data)
dados_novos

In [ ]:
classes = dados_novos['class'].unique()

figure = plt.figure(figsize=(20,10))
axis = figure.add_subplot(111, projection='3d')

for classe in classes:
    valores =  dados_novos[dados_novos['class'] == classe].values
    x = valores[:,0]
    y = valores[:,1]
    z = valores[:,2]
    axis.scatter(x,y,z, label=classe)
    
axis.legend()

In [ ]:
X = dados_novos[['x1','x2','x3']].values
y = dados_novos[['class']].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3)
X_train.shape, X_test.shape

In [ ]:
model = Perceptron()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
w = np.random.randn(8 , X.shape[1]) # [8x3] 8 caracteristica para 3 colunas de entrada
learning_rate = 1e-3

for step in range(201):
    cost = 0
    for x_n, y_n in zip(X_train,y_train):
        y_pred = np.dot(x_n,w.T)
        y_pred = np.argmax(y_pred)
        error = y_n - y_pred
        #atualizar os pesos, se ele errou, quando y_n é diferente de y_pred (por isso atualizar os pesos nas posições respectivas)
        if not (y_n == y_pred):
            w[int(y_n)] +=  learning_rate* x_n # adiciona importancia a entrada
            w[int(y_pred)] -=  learning_rate* x_n #  diminui a importancia pois errou 

        cost += error**2
    if step % 10 == 0:
        print('epoch: {0}/{1} loss_train: {2:.4f}'.format(step,200,0.5*(cost/(len(X_train)))))

print('w: ',w)

In [ ]:
y_pred = np.argmax( np.dot(X_test, w.T) , axis=1 )

print(classification_report(y_test,y_pred))

2) Implemente uma rede perceptron de múltiplas camadas e utilize-a para aproximar as
funções abaixo. Em seguida, compare os resultados com as curvas exatas. No caso das letras
(b) e (c), apresente também a curva do erro médio de treinamento com relação ao número de
épocas e a curva do erro médio com o conjunto de validação.

    a) a função lógica XOR

    b) f(x) = log10(x), onde 1 ≤ x ≤ 10

    c) f(x) = 10x^5+ 5x^4+ 2x^3– 0.5x^2+ 3x + 2, onde 0 ≤ x ≤ 5


A) Função lógica XOR

In [ ]:
def gerar_dados_xor(quantidade, base):
    dados_gerados = deepcopy(base)
    valores = dados_gerados[['x0','x1']].values
    
    for i in range(quantidade):
        ruido = np.array([np.random.uniform(-0.1,0.1) for i in range(x.shape[0]*x.shape[1])]).reshape(4,2)
        novos_valores = valores + ruido
        for k in range(len(novos_valores)):
            #adicionando novos dados no final do dataset com o ruido
            if (k == 2):
                dados_gerados.loc[len(dados_gerados)] = [novos_valores[k][0],novos_valores[k][1],1]
            elif (k == 3):
                dados_gerados.loc[len(dados_gerados)] = [novos_valores[k][0],novos_valores[k][1],0]
            else:
                dados_gerados.loc[len(dados_gerados)] = [novos_valores[k][0],novos_valores[k][1],k]
            
            
    return dados_gerados

In [ ]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]).reshape(-1, 1)

print(x.shape, y.shape)
plt.scatter(x[:,0], x[:,1], c=list(np.array(y).ravel()), s=15, cmap='bwr')

In [ ]:
data  = pd.DataFrame(np.column_stack((x,y)), columns=["x0","x1","y"])
novos_dados = gerar_dados_xor(500, data)

In [ ]:
classes = novos_dados['y'].unique()

figure = plt.figure(figsize=(20,10))
axis = figure.add_subplot(111)

for classe in classes:
    valores =  novos_dados[novos_dados['y'] == classe].values
    x = valores[:,0]
    y = valores[:,1]
    axis.scatter(x,y, label=classe)

In [ ]:
x = novos_dados[["x0","x1"]].values
y = novos_dados[["y"]].values
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= 0.3)

x.shape, y.shape # mostrando a quantidade de dados utilizados.

In [ ]:
# Utilizando sklearn
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='relu', max_iter=10000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
# print('Predições:', y_pred, sep='\n')
print('Acurácia: {:.2f}%'.format(100*accuracy_score(y_test, y_pred > 0.5)))

classification_predictions(X_test, y_pred, is_binary=True, threshold=0.5, nn=model, cmap='bwr')

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(model.loss_curve_)
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento'])
plt.show()

In [ ]:
# Rede Neural implementada

input_dim, output_dim = X_train.shape[1], y_train.shape[1]

nn = NeuralNetwork(cost_func = binary_cross_entropy, learning_rate= 1e-1)
nn.layers.append(Layer(input_dim=input_dim, output_dim=10, activation=sigmoid))
nn.layers.append(Layer(input_dim=10, output_dim=output_dim, activation=sigmoid))

nn.fit(X_train,y_train, X_test, y_test,True, epochs=3000, verbose=300)

y_pred = nn.predict(x)
print('Predições:', y_pred, sep='\n')
print('Acurácia: {:.2f}%'.format(100*accuracy_score(y, y_pred > 0.5)))

classification_predictions(x, y, is_binary=True, threshold=0.5, nn=nn, cmap='bwr')

In [ ]:
plt.plot(nn.train_val_loss['loss'])
plt.plot(nn.train_val_loss['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

pred = np.round(nn.predict(X_test))
pred_train = np.round(nn.predict(X_train))

print(f'--> Acuracia (train): {accuracy_score(y_train, pred_train):.4f}')
print(f'--> Acuracia (test): {accuracy_score(y_test, pred):.4f}')

B) f(x) = log10(x), onde 1 ≤ x ≤ 10

In [ ]:
x,y = make_log10(100, 1, 10, noise=0, random_state=None)

In [ ]:
# normalizar os dados
minmax = MinMaxScaler(feature_range=(-1, 1))
x = minmax.fit_transform(x.astype(np.float64))

x.shape, y.shape # mostrando a quantidade de dados utilizados.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= 0.3)

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor( max_iter=3000).fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(regr.loss_curve_)
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento'])
plt.show()

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(1)))
model.add(tf.keras.layers.Dense(10, activation='tanh'))
model.add(tf.keras.layers.Dense(10, activation='tanh'))
model.add(tf.keras.layers.Dense(1,activation='linear'))

# Compilar o modelo
model.compile(loss='mean_squared_error', metrics='mean_squared_error', optimizer='adam')

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

pred = np.round(model.predict(X_test))
pred_train = np.round(model.predict(X_train))

In [ ]:
## Utilizando a Rede Neural Implementada
input_dim, output_dim = X_train.shape[1], y_train.shape[1]


nn = NeuralNetwork(cost_func = mse, learning_rate=1e-2)
nn.layers.append(Layer(input_dim=input_dim, output_dim= 10, activation = tanh))
nn.layers.append(Layer(input_dim=10, output_dim= output_dim, activation = linear))

nn.fit(X_train,y_train, X_test, y_test, True, epochs=3000, verbose= 300)

In [ ]:
plt.plot(nn.train_val_loss['loss'])
plt.plot(nn.train_val_loss['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

pred = np.round(nn.predict(X_test))
pred_train = np.round(nn.predict(X_train))

In [ ]:
input_dim, output_dim = x.shape[1], y.shape[1]


nn = NeuralNetwork(cost_func = mse, learning_rate=1e-2)
nn.layers.append(Layer(input_dim=input_dim, output_dim= 10, activation = tanh))
nn.layers.append(Layer(input_dim=10, output_dim= output_dim, activation = linear))

nn.fit(x,y, epochs=3000, verbose= 300)

plt.figure(figsize = (15,10))
plt.subplot(1, 2, 1)
plt.scatter(x,y)
plt.plot(x, nn.predict(x), c="green")

C) f(x) = 10x^5+ 5x^4+ 2x^3– 0.5x^2+ 3x + 2, onde 0 ≤ x ≤ 5

In [ ]:
x = np.linspace(0,1,100)
y = 10*x**5 + 5*x**4 + 2*x**3 - 0.5*x**2 + 3*x + 2 

minmax = MinMaxScaler(feature_range=(-1, 1))
x = minmax.fit_transform(x.reshape(-1,1).astype(np.float64))

# x = x.reshape(-1,1)
y = y.reshape(-1,1)
x.shape, y.shape # mostrando a quantidade de dados utilizados.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size= 0.3)

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor( max_iter=3000).fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(regr.loss_curve_)
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento'])
plt.show()

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(1)))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='linear'))

# Compilar o modelo
model.compile(loss='mean_squared_error', metrics='mean_squared_error', optimizer='sgd')

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

In [ ]:
input_dim, output_dim = X_train.shape[1], y_train.shape[1]

nn = NeuralNetwork(cost_func = mse, learning_rate = 1e-3)
nn.layers.append(Layer(input_dim=input_dim, output_dim=12, activation=relu))
nn.layers.append(Layer(input_dim=12, output_dim=12, activation=relu))
nn.layers.append(Layer(input_dim=12, output_dim=output_dim, activation=linear))

nn.fit(X_train,y_train, X_test, y_test, True, epochs=3000, verbose=300)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento', 'Validacao'])
plt.show()

In [ ]:
input_dim, output_dim = x.shape[1], y.shape[1]

nn = NeuralNetwork(cost_func = mse, learning_rate = 1e-3)
nn.layers.append(Layer(input_dim=input_dim, output_dim=12, activation=relu))
nn.layers.append(Layer(input_dim=12, output_dim=12, activation=relu))
nn.layers.append(Layer(input_dim=12, output_dim=output_dim, activation=linear))

nn.fit(x,y, epochs=3000, verbose=300)


plt.figure(figsize = (15,10))
plt.subplot(1, 2, 1)
plt.scatter(x,y)
plt.plot(x, nn.predict(x), c="green")

## Questão 3

Considere um problema de classificação de padrões constituído de oito padrões. A
distribuição dos padrões forma um círculo centrado na origem de raio unitário e contido no
círculo um losango também centrado na origem e com lados iguais à raiz de 2. Os dados das
classes C1, C2, C3, C4 correspondem aos quatro setores do losango e as outras quatro
classes correspondem aos setores delimitados pelo círculo e os lados do losango. Após gerar
aleatoriamente dados que venham formar estas distribuições de dados, selecione um conjunto
de treinamento e um conjunto de validação. Treine duas redes perceptron (uma rede
utilizando a regra delta convencional, e outra usando a regra delta com termo do
momento), para classificar os padrões associados a cada uma das classes. Verifique o
desempenho dos classificadores usando o conjunto de validação e calculando a matriz de
confusão

In [ ]:
def Generate_data_problem(n_exemplos):
    x = np.random.uniform(-1,1,n_exemplos)
    y = np.random.uniform(-1,1,n_exemplos)
    
    dados = pd.DataFrame(data={'x':[0.1], 'y': [0.1], 'Class': [0]})
    
    
    for i in range(n_exemplos):
        if ( y[i] <= 1 - x[i] and x[i] >= 0 and y[i] >= 0 ):
            dados.loc[len(dados)] = [x[i],y[i],0]
            
        elif ( y[i] <= x[i] + 1 and x[i] <= 0 and y[i]>=0 ):
            dados.loc[len(dados)] = [x[i],y[i],1]
            
        elif ( y[i] >= -x[i] - 1 and x[i] <= 0 and y[i] <= 0 ):
            dados.loc[len(dados)] = [x[i],y[i],2]
            
        elif ( y[i] >= x[i] - 1 and x[i] >= 0 and y[i] <= 0 ):
            dados.loc[len(dados)] = [x[i],y[i],3]
            
        elif ( y[i] >= 1 - x[i] and x[i] >= 0 and y[i] >= 0  and x[i]**2 + y[i]**2 <=1 ):
            dados.loc[len(dados)] = [x[i],y[i],4]
            
        elif ( y[i] >= x[i] + 1 and x[i] <= 0 and y[i]>=0 and x[i]**2 + y[i]**2 <=1 ):
            dados.loc[len(dados)] = [x[i],y[i],5]
            
        elif ( y[i] <= -x[i] - 1 and x[i] <= 0 and y[i] <= 0 and x[i]**2 + y[i]**2 <=1 ):
            dados.loc[len(dados)] = [x[i],y[i],6]
            
        elif ( y[i] <= x[i] - 1 and x[i] >= 0 and y[i] <= 0 and x[i]**2 + y[i]**2 <=1 ):
            dados.loc[len(dados)] = [x[i],y[i],7]
            
            
    return dados

In [ ]:
dados_gerados = Generate_data_problem(5000)

In [ ]:
classes = dados_gerados['Class'].unique()
    
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111)

for classe in classes:
    valores = dados_gerados[dados_gerados['Class'] == classe].values
    x,y = valores[:,0] , valores[:,1]
    ax.scatter(x,y, label=classe)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dados_gerados[['x','y']].values,dados_gerados[['Class']].values, test_size=0.3)


dados_gerados[['x','y']].shape, dados_gerados[['Class']].shape # mostrando a quantidade de dados utilizados.

In [ ]:
one = OneHotEncoder(sparse=False)

y_train = one.fit_transform(y_train.reshape(-1,1))
y_train

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor( max_iter=3000).fit(X_train, y_train)

y_pred = regr.predict(X_test)
one.inverse_transform(y_pred)
pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test,pred))
print('Matriz de Confusão: \n')
print(confusion_matrix(y_test,pred))

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(regr.loss_curve_)
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento'])
plt.show()

In [ ]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor( max_iter=3000, momentum=0.52).fit(X_train, y_train)

y_pred = regr.predict(X_test)
one.inverse_transform(y_pred)
pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test,pred))
print('Matriz de Confusão: \n')
print(confusion_matrix(y_test,pred))

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(regr.loss_curve_)
plt.title('Metrica de erro')
plt.ylabel('Erro')
plt.xlabel('Epoca')
plt.legend(['Treinamento'])
plt.show()

In [ ]:
input_dim, output_dim = X_train.shape[1], y_train.shape[1]

nn = NeuralNetwork(cost_func = softmax_neg_log_likelihood, learning_rate = 1e-1)

nn.layers.append(Layer(input_dim=input_dim, output_dim=10, activation= tanh))
nn.layers.append(Layer(input_dim=10, output_dim=10, activation= relu))
nn.layers.append(Layer(input_dim=10, output_dim=output_dim, activation= linear))

nn.fit(X_train,y_train, epochs=3000, verbose=300)

y_pred = nn.predict(X_test)

In [ ]:
one.inverse_transform(y_pred)

In [ ]:
pred = np.argmax(y_pred, axis=1)


print(classification_report(y_test,pred))

print('Matriz de Confusão: \n')
print(confusion_matrix(y_test,pred))

In [ ]:
y_pred = nn.predict(X_test[0])
np.argmax(y_pred)

In [ ]:
X_test[0]